In [25]:
import numpy as np
import pandas as pd
import requests
import json
import spotipy
from IPython.display import display

### Set credentials needed to authorise APIs

In [26]:
with open('../auth.json') as f:
    credentials = json.load(f)

setlistfm_api_key = credentials['setlistfm']['api_key']
spotify_client_id = credentials['spotify']['client_id']
spotify_client_secret = credentials['spotify']['client_secret']

### Setlist FM

#### Plan of action
1. Given a lineup (list of band names), get their Musicbrainz identifiers (`mbid`) via `https://api.setlist.fm/rest/1.0/search/artists`
2. Retrieve the setlists for each artist using their `mbid` via `https://api.setlist.fm/rest/1.0/artist/{artist_mbid}/setlists
`

In [46]:
lineup = pd.read_csv(
    '../data/test_lineup.csv', header=None, names=['band'], encoding="ISO-8859-1"
)['band'].values

In [47]:
len(lineup)

114

In [29]:
lineup

array(['Gorillaz', 'The Strokes', 'Young Thug', 'Tame Impala',
       'Disclosure', 'Beck', 'Tyler, The Creator', 'The National',
       'Charli XCX', 'Bad Bunny', 'Iggy Pop', 'Jorja Smith', 'Metronomy',
       'Earl Sweatshirt', 'Caribou', 'Brockhampton', 'Jamie xx',
       'DJ Shadow', 'Kurt Vile', 'Yo La Tengo', 'FKA twigs',
       'Cigarettes After Sex', 'The Jesus and Mary Chain', 'Dinosaur Jr.',
       'King Krule', 'Chromatics', 'Pavement', 'Young Dolph',
       'Black Lips', 'Mabel', 'Freddie Gibbs', 'Khruangbin',
       'King Gizzard & The Lizard Wizard', 'DIIV', 'Rapsody', 'Madlib',
       'Bauhaus', 'Kano', 'Black Coffee', 'Desire', 'Little Simz',
       'Jamila Woods', 'IDLES', 'Autechre', 'Slowthai', 'C. Tangana',
       'Nina Kraviz', 'Pabllo Vittar', 'Napalm Death', 'Les Savy Fav',
       'Beach Bunny', 'Koffee', 'Rolling Blackouts Coastal Fever',
       'Big Freedia', 'Einst\x9frzende Neubauten', 'Shellac', 'Duki',
       'Mano Le Tough', 'Leon Vynehall', 'Shame', 'Fati

In [30]:
lineup = lineup[:10]

In [31]:
lineup

array(['Gorillaz', 'The Strokes', 'Young Thug', 'Tame Impala',
       'Disclosure', 'Beck', 'Tyler, The Creator', 'The National',
       'Charli XCX', 'Bad Bunny'], dtype=object)

In [32]:
artists_url = 'https://api.setlist.fm/rest/1.0/search/artists'

In [33]:
lineup_mbids = []
not_found = []

for name in lineup:
    req = requests.get(artists_url,
                       headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                       params={'artistName': name, 'p': 1, 'sort': 'relevance'}
    )
    
    i = 0
    while (not req.ok) & (i <= 3):
        req = requests.get(artists_url,
                           headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                           params={'artistName': name, 'p': 1, 'sort': 'relevance'}
        )
        i += 1
    
    if req.ok:
        artist_response = req.json()['artist']
        num_artists = len(artist_response)
        if num_artists > 1:
            for i in range(num_artists):
                if artist_response[i]['name'].lower() == name.lower():
                    mbid = artist_response[i]['mbid']
                    lineup_mbids.append({'name': name, 'mbid': mbid})
                    break
                    
    else:
        print(f'Couldn\'t find {name}')
        not_found.append(name)

In [34]:
lineup_mbids[-10:]

[{'name': 'Gorillaz', 'mbid': 'e21857d5-3256-4547-afb3-4b6ded592596'},
 {'name': 'The Strokes', 'mbid': 'f181961b-20f7-459e-89de-920ef03c7ed0'},
 {'name': 'Young Thug', 'mbid': '800760de-bdf8-43a2-8fe0-44a2401a5515'},
 {'name': 'Tame Impala', 'mbid': '63aa26c3-d59b-4da4-84ac-716b54f1ef4d'},
 {'name': 'Disclosure', 'mbid': 'ae65c507-d9a0-4d42-9a6c-2b1f82158b9f'},
 {'name': 'Beck', 'mbid': '309c62ba-7a22-4277-9f67-4a162526d18a'},
 {'name': 'Tyler, The Creator',
  'mbid': 'f6beac20-5dfe-4d1f-ae02-0b0a740aafd6'},
 {'name': 'The National', 'mbid': '664c3e0e-42d8-48c1-b209-1efca19c0325'},
 {'name': 'Charli XCX', 'mbid': '260b6184-8828-48eb-945c-bc4cb6fc34ca'},
 {'name': 'Bad Bunny', 'mbid': '89aa5ecb-59ad-46f5-b3eb-2d424e941f19'}]

In [35]:
not_found

[]

In [36]:
artist_setlist = []

for a in lineup_mbids:
    songs_played = []
    mbid = a['mbid']
    setlists_url = f'https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists'
    
    req = requests.get(setlists_url,
             headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
             params={'p': 1}
    )
    
    i = 0
    while (not req.ok) & (i <= 3):
        req = requests.get(setlists_url,
                 headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                 params={'p': 1}
        )
        i += 1
            
    if req.ok:
    
        setlist_response = req.json()['setlist']
        num_setlists = len(setlist_response)

        for i in range(num_setlists):
            setlist = setlist_response[i]['sets']['set']
            num_sections = len(setlist)
            total_songs = []
            for p in range(num_sections):
                total_songs += setlist[p]['song']
            num_songs = len(total_songs)

            for i in range(num_songs):
                song = total_songs[i]
                song_title = song['name']
                # if the song is a cover add the original artist to the song title
                if 'cover' in song:
                    song_title += ' {}'.format(song['cover']['name'])
                songs_played.append(song_title)
                                           
        most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)

        artist_setlist.append({
            'artist': a['name'],
            'setlist': most_played_songs
        })
    else:
        print(a['name'])

In [37]:
artist_setlist

[{'artist': 'Gorillaz',
  'setlist': ['On Melancholy Hill',
   'Humility',
   'Tomorrow Comes Today',
   'Every Planet We Reach Is Dead',
   'Tranz',
   'Last Living Souls',
   'Souk Eye',
   'Andromeda',
   'M1 A1',
   'El Mañana',
   'Kids With Guns',
   'Strobelite',
   'Stylo',
   'Feel Good Inc.',
   'Clint Eastwood']},
 {'artist': 'The Strokes',
  'setlist': ['Hard to Explain',
   'You Only Live Once',
   'Someday',
   'Heart in a Cage',
   'Reptilia',
   'The Modern Age',
   'Last Nite',
   'What Ever Happened?',
   'New York City Cops',
   'Ize of the World',
   'Soma',
   'Juicebox',
   "I Can't Win",
   'Meet Me in the Bathroom',
   'On the Other Side']},
 {'artist': 'Young Thug',
  'setlist': ['Best Friend',
   'With That',
   'The London',
   'Digits',
   'Pick Up the Phone Travis Scott & Young Thug',
   'With Them',
   'Ecstasy',
   'Check',
   'Killed Before Future & Young Thug',
   'About the Money T.I.',
   'Relationship',
   'On the Run',
   'Hot',
   'Lifestyle Rich G

### Spotify

In [38]:
username = 'adrialuz'
scope = 'playlist-modify-public'

token = spotipy.util.prompt_for_user_token(username, scope)

sp = spotipy.Spotify(auth=token)

sp.trace = False

Get the URI codes for each track

In [39]:
uris = []
missing_songs = []

for a in artist_setlist:
    artist = a['artist']
    setlist = a['setlist']
    for s in setlist:
        s = s.replace(',', '').replace('\'', '').replace('"', '').replace('.', '').replace(
            '?', '').replace(')', '').replace('(', '').replace('/', '').replace(
            '\\', '').replace('&', '').replace('-', '')
        items = sp.search(q=f'artist:{artist} track:{s}', limit=1)['tracks']['items']
        if items:
            uri = items[0]['id']
            uris.append(uri)
        else:
            items = sp.search(q=f'track:{s}', limit=1)['tracks']['items']
            if items:
                if items != [None]:
                    uri = items[0]['id']
                    uris.append(uri)
            else:
                missing_songs.append({
                    'artist': artist,
                    'song': s
                })

In [40]:
len(uris)

147

In [41]:
len(missing_songs)

3

In [42]:
missing_songs

[{'artist': 'Young Thug', 'song': 'Killed Before Future  Young Thug'},
 {'artist': 'Tame Impala', 'song': 'Mutant Gossip'},
 {'artist': 'Charli XCX', 'song': 'Track 10  Blame It on Your Love'}]

In [43]:
divisor = int(np.floor(len(uris) / np.ceil(len(uris) / 100)))
divisor

73

In [44]:
times = int(np.floor(len(uris) / divisor))
times

2

In [45]:
for i in range(times):
    subset = uris[divisor*i:divisor*(i+1)]
    sp.user_playlist_add_tracks(username, playlist_id='5sfuRJ3UgOv35RizTPycAC',
                            tracks=subset)